<a href="https://colab.research.google.com/github/Chenyuju/chen/blob/master/KNHANES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [ ]:
hn16_all = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load("/content/sample_data/hn16_base.csv",header=True);

hn16_all.show()

In [ ]:
hn16_all.columns

In [ ]:
from pyspark.sql.functions import isnan, isnull, when, count, col

hn16_all.select([count(when(isnull(c), c)).alias(c) for c in hn16_all.columns]).show()

In [ ]:
data = pd.read_csv('/content/sample_data/hn16_base.csv')
print(data.head(7))

In [ ]:
def msv1(data, thresh=20, color='black', edgecolor='black', width=15, height=3):
    plt.figure(figsize=(width,height))
    percentage=(data.isnull().mean())*100
    percentage.sort_values(ascending=False).plot.bar(color=color, edgecolor=edgecolor)
    plt.axhline(y=thresh, color='r', linestyle='-')
    plt.title('Missing values percentage per column', fontsize=10)
    plt.text(len(data.isnull().sum()/len(data))/1.7, thresh+12.5, 'Columns with more than %s%s missing values' %(thresh, '%'), fontsize=12,weight='bold', color='crimson',
         ha='left' ,va='top')
    plt.text(len(data.isnull().sum()/len(data))/1.7, thresh - 5, 'Columns with less than %s%s missing values' %(thresh, '%'), fontsize=12,weight='bold', color='blue',
         ha='left' ,va='top')
    plt.xlabel('Columns', size=15)
    plt.ylabel('Missing values percentage')

    return plt.show()

In [ ]:
msv1(data, 30, color=('silver', 'gainsboro', 'lightgreen', 'white', 'lightpink'))

In [ ]:
percentage=(data.isnull().mean())*100
pd.set_option("max_rows", None)
percentage.sort_values(ascending=False)

In [ ]:
df=data.dropna(thresh=2445, axis=1)
print(f"Data shape before cleaning {data.shape}")
print(f"Data shape after cleaning {df.shape}")
print(f"We dropped {data.shape[1]- df.shape[1]} columns")

In [ ]:
percentage=(df.isnull().mean())*100
pd.set_option("max_rows", None)
percentage.sort_values(ascending=False)

In [ ]:
df['N_EN'].count()

In [ ]:
lst_12 = [1, 2]
for i in range((df['N_EN']).count()):
  if ((df['age'] == 1) | (df['age'] == 2)).all():
    df['diff_EN'] = np.where(df['N_EN'] == 1000, 0, df['N_EN'] - 1000)
  elif ((df['age'] == 3) | (df['age'] == 4) | (df['age'] == 5)).all():
    df['diff_EN'] = np.where(df['N_EN'] == 1400, 0, df['N_EN'] - 1400)
  elif ((df['sex'] == 1).all()) & ((df['age'] >= 6) & (df['age'] <=8)).any():
    df['diff_EN'] = np.where(df['N_EN'] == 1700, 0, df['N_EN'] - 1700)
  elif (df['sex'] == '1').all() & (df['age'] >= 9).any() & (df['age'] <=11).any():
    df['diff_EN'] = np.where(df['N_EN'] == 2100, 0, df['N_EN'] - 2100)
  elif (df['sex'] == '1').all() & (df['age'] >= 12).any() & (df['age'] <=14).any():
    df['diff_EN'] = np.where(df['N_EN'] == 2500, 0, df['N_EN'] - 2500)
  elif (df['sex'] == '1').all() & (df['age'] >= 15).any() & (df['age'] <=18).any():
    df['diff_EN'] = np.where(df['N_EN'] == 2700, 0, df['N_EN'] - 2700)
  elif (df['sex'] == '1').all() & (df['age'] >= 19).any() & (df['age'] <=29).any():
    df['diff_EN'] = np.where(df['N_EN'] == 2600, 0, df['N_EN'] - 2600)
  elif (df['sex'] == '1').all() & (df['age'] >= 30).any() & (df['age'] <=49).any():
    df['diff_EN'] = np.where(df['N_EN'] == 2400, 0, df['N_EN'] - 2400)
  elif (df['sex'] == '1').all() & (df['age'] >= 50).any() & (df['age'] <=64).any():
    df['diff_EN'] = np.where(df['N_EN'] == 2200, 0, df['N_EN'] - 2200)
  elif (df['sex'] == '1').all() & (df['age'] >= 65).any() & (df['age'] <=74).any():
    df['diff_EN'] = np.where(df['N_EN'] == 2000, 0, df['N_EN'] - 2000)
  elif (df['sex'] == '1').all() & (df['age'] >= 75).any():
    df['diff_EN'] = np.where(df['N_EN'] == 2000, 0, df['N_EN'] - 2000)
  else:
    df['diff_EN'] = df['N_EN'].copy()

print(df[['age', 'sex', 'N_EN', 'diff_EN']])

In [ ]:
low_limit = pd.Series([1, 3, 6, 9, 12, 15, 19, 30, 50, 65, 75, np.nan], index=df)
high_limit = pd.Series([2, 5, 8, 11, 14, 18, 29, 49, 64, 74, 120], index=df['age'])

In [ ]:
SN = ((df < high_limit) & (df > low_limit)).all(axis=1)

In [ ]:
age_12 = ['1', '2']
for i in range((df['N_EN']).count()):
  if df.loc[df['age'].isin(age_12)].any():
    df['diff_EN'] = 'aaaaaaa'

In [ ]:
# --- 나이와 성별에 대한 14 가지의 표준영양소값 구하기 ---

In [ ]:
def standard_EN(series):
    age = series["age"]
    sex = series["sex"]

    if ((age >= 1) & (age < 3)):
        return 1000
    elif ((age >= 3) & (age < 6)):
        return 1400
    elif (sex == 1):
        if ((age >= 6) & (age < 9)):
            return 1700
        elif ((age >= 9) & (age < 12)):
            return 2100
        elif ((age >= 12) & (age < 15)):
            return 2500
        elif ((age >= 15) & (age < 19)):
            return 2700
        elif ((age >= 19) & (age < 30)):
            return 2600
        elif ((age >= 30) & (age < 40)):
            return 2400
        elif ((age >= 40) & (age < 65)):
            return 2200
        elif ((age >= 65) & (age < 75)):
            return 2000
        elif (age >= 75):
            return 2000
    elif (sex == 2):
        if ((age >= 6) & (age < 9)):
            return 1500
        elif ((age >= 9) & (age < 12)):
            return 1800
        elif ((age >= 12) & (age < 15)):
            return 2000
        elif ((age >= 15) & (age < 19)):
            return 2100
        elif ((age >= 19) & (age < 30)):
            return 1900
        elif ((age >= 30) & (age < 40)):
            return 1800
        elif ((age >= 40) & (age < 65)):
            return 1600
        elif ((age >= 65) & (age < 75)):
            return 2000
        elif (age >= 75):
            return 1600
    else:
      return 0

In [ ]:
df['SN_EN'] = df.apply(standard_EN, axis=1)

In [ ]:
def standard_PROT(series):
    age = series["age"]
    sex = series["sex"]

    if ((age >= 1) & (age < 3)):
        return 15
    elif ((age >= 3) & (age < 6)):
        return 20
    elif (sex == 1):
        if ((age >= 6) & (age < 9)):
            return 30
        elif ((age >= 9) & (age < 12)):
            return 40
        elif ((age >= 12) & (age < 15)):
            return 55
        elif ((age >= 15) & (age < 19)):
            return 65
        elif ((age >= 19) & (age < 30)):
            return 65
        elif ((age >= 30) & (age < 40)):
            return 60
        elif ((age >= 40) & (age < 65)):
            return 60
        elif ((age >= 65) & (age < 75)):
            return 55
        elif (age >= 75):
            return 55
    elif (sex == 2):
        if ((age >= 6) & (age < 9)):
            return 25
        elif ((age >= 9) & (age < 12)):
            return 40
        elif ((age >= 12) & (age < 15)):
            return 50
        elif ((age >= 15) & (age < 19)):
            return 50
        elif ((age >= 19) & (age < 30)):
            return 55
        elif ((age >= 30) & (age < 40)):
            return 50
        elif ((age >= 40) & (age < 65)):
            return 50
        elif ((age >= 65) & (age < 75)):
            return 45
        elif (age >= 75):
            return 45
    else:
      return 0

In [ ]:
df['SN_PROT'] = df.apply(standard_PROT, axis=1)

In [ ]:
def standard_TDF(series):
    age = series["age"]
    sex = series["sex"]

    if ((age >= 1) & (age < 3)):
        return 10
    elif ((age >= 3) & (age < 6)):
        return 15
    elif (sex == 1):
        if ((age >= 6) & (age < 9)):
            return 20
        elif ((age >= 9) & (age < 12)):
            return 20
        elif ((age >= 12) & (age < 75)):
            return 25
        elif (age >= 75):
            return 55
    elif (sex == 2):
        if ((age >= 6) & (age < 75)):
            return 20
        elif (age >= 75):
            return 20
    else:
      return 0

In [ ]:
df['SN_TDF'] = df.apply(standard_TDF, axis=1)

In [ ]:
def standard_WATER(series):
    age = series["age"]
    sex = series["sex"]

    if ((age >= 1) & (age < 3)):
        return 1100
    elif ((age >= 3) & (age < 6)):
        return 1500
    elif (sex == 1):
        if ((age >= 6) & (age < 9)):
            return 1800
        elif ((age >= 9) & (age < 12)):
            return 2100
        elif ((age >= 12) & (age < 15)):
            return 2300
        elif ((age >= 15) & (age < 19)):
            return 2600
        elif ((age >= 19) & (age < 30)):
            return 2600
        elif ((age >= 30) & (age < 40)):
            return 2500
        elif ((age >= 40) & (age < 65)):
            return 2200
        elif ((age >= 65) & (age < 75)):
            return 2100
        elif (age >= 75):
            return 2100
    elif (sex == 2):
        if ((age >= 6) & (age < 9)):
            return 1700
        elif ((age >= 9) & (age < 12)):
            return 1900
        elif ((age >= 12) & (age < 19)):
            return 2000
        elif ((age >= 19) & (age < 30)):
            return 2100
        elif ((age >= 30) & (age < 40)):
            return 2000
        elif ((age >= 40) & (age < 65)):
            return 1900
        elif ((age >= 65) & (age < 75)):
            return 1800
        elif (age >= 75):
            return 1800
    else:
      return 0

In [ ]:
df['SN_WATER'] = df.apply(standard_WATER, axis=1)

In [ ]:
def standard_VITC(series):
    age = series["age"]
    sex = series["sex"]

    if ((age >= 1) & (age < 3)):
        return 35
    elif ((age >= 3) & (age < 6)):
        return 40
    elif (sex == 1):
        if ((age >= 6) & (age < 9)):
            return 55
        elif ((age >= 9) & (age < 12)):
            return 70
        elif ((age >= 12) & (age < 15)):
            return 90
        elif ((age >= 15) & (age < 19)):
            return 105
        elif ((age >= 19) & (age < 75)):
            return 100
        elif (age >= 75):
            return 100
    elif (sex == 2):
        if ((age >= 6) & (age < 9)):
            return 60
        elif ((age >= 9) & (age < 12)):
            return 80
        elif ((age >= 12) & (age < 19)):
            return 100
        elif ((age >= 19) & (age < 30)):
            return 95
        elif ((age >= 30) & (age < 75)):
            return 100
        elif (age >= 75):
            return 100
    else:
      return 0

In [ ]:
df['SN_VITC'] = df.apply(standard_VITC, axis=1)

In [ ]:
def standard_B1(series):
    age = series["age"]
    sex = series["sex"]

    if ((age >= 1) & (age < 3)):
        return 0.5
    elif ((age >= 3) & (age < 6)):
        return 0.5
    elif (sex == 1):
        if ((age >= 6) & (age < 9)):
            return 0.7
        elif ((age >= 9) & (age < 12)):
            return 0.9
        elif ((age >= 12) & (age < 15)):
            return 1.1
        elif ((age >= 15) & (age < 19)):
            return 1.3
        elif ((age >= 19) & (age < 75)):
            return 1.2
        elif (age >= 75):
            return 1.2
    elif (sex == 2):
        if ((age >= 6) & (age < 9)):
            return 0.7
        elif ((age >= 9) & (age < 12)):
            return 0.9
        elif ((age >= 12) & (age < 14)):
            return 1.1
        elif ((age >= 15) & (age < 18)):
            return 1.2
        elif ((age >= 19) & (age < 75)):
            return 1.1
        elif (age >= 75):
            return 1.1
    else:
      return 0

In [ ]:
df['SN_B1'] = df.apply(standard_B1, axis=1)

In [ ]:
def standard_B2(series):
    age = series["age"]
    sex = series["sex"]

    if ((age >= 1) & (age < 3)):
        return 0.5
    elif ((age >= 3) & (age < 6)):
        return 0.6
    elif (sex == 1):
        if ((age >= 6) & (age < 9)):
            return 0.9
        elif ((age >= 9) & (age < 12)):
            return 1.2
        elif ((age >= 12) & (age < 14)):
            return 1.5
        elif ((age >= 15) & (age < 18)):
            return 1.7
        elif ((age >= 19) & (age < 75)):
            return 1.5
        elif (age >= 75):
            return 1.5
    elif (sex == 2):
        if ((age >= 6) & (age < 9)):
            return 0.8
        elif ((age >= 9) & (age < 12)):
            return 1.0
        elif ((age >= 12) & (age < 75)):
            return 1.2
        elif (age >= 75):
            return 1.2
    else:
      return 0

In [ ]:
df['SN_B2'] = df.apply(standard_B2, axis=1)

In [ ]:
def standard_NIAC(series):
    age = series["age"]
    sex = series["sex"]

    if ((age >= 1) & (age < 3)):
        return 6
    elif ((age >= 3) & (age < 6)):
        return 7
    elif (sex == 1):
        if ((age >= 6) & (age < 9)):
            return 9
        elif ((age >= 9) & (age < 12)):
            return 12
        elif ((age >= 12) & (age < 15)):
            return 15
        elif ((age >= 15) & (age < 19)):
            return 17
        elif ((age >= 19) & (age < 75)):
            return 16
        elif (age >= 75):
            return 16
    elif (sex == 2):
        if ((age >= 6) & (age < 9)):
            return 9
        elif ((age >= 9) & (age < 12)):
            return 12
        elif ((age >= 12) & (age < 15)):
            return 15
        elif ((age >= 15) & (age < 75)):
            return 14
        elif (age >= 75):
            return 14
    else:
      return 0

In [ ]:
df['SN_NIAC'] = df.apply(standard_NIAC, axis=1)

In [ ]:
def standard_VA_RAE(series):
    age = series["age"]
    sex = series["sex"]

    if ((age >= 1) & (age < 3)):
        return 300
    elif ((age >= 3) & (age < 6)):
        return 350
    elif (sex == 1):
        if ((age >= 6) & (age < 9)):
            return 450
        elif ((age >= 9) & (age < 12)):
            return 600
        elif ((age >= 12) & (age < 15)):
            return 750
        elif ((age >= 15) & (age < 19)):
            return 850
        elif ((age >= 19) & (age < 30)):
            return 800
        elif ((age >= 30) & (age < 65)):
            return 750
        elif ((age >= 65) & (age < 75)):
            return 700
        elif (age >= 75):
            return 700
    elif (sex == 2):
        if ((age >= 6) & (age < 9)):
            return 400
        elif ((age >= 9) & (age < 12)):
            return 550
        elif ((age >= 12) & (age < 15)):
            return 650
        elif ((age >= 15) & (age < 19)):
            return 600
        elif ((age >= 19) & (age < 50)):
            return 650
        elif ((age >= 50) & (age < 65)):
            return 600
        elif ((age >= 65) & (age < 75)):
            return 550
        elif (age >= 75):
            return 550
    else:
      return 0

In [ ]:
df['SN_VA_RAE'] = df.apply(standard_VA_RAE, axis=1)

In [ ]:
def standard_CA(series):
    age = series["age"]
    sex = series["sex"]

    if ((age >= 1) & (age < 3)):
        return 500
    elif ((age >= 3) & (age < 6)):
        return 600
    elif (sex == 1):
        if ((age >= 6) & (age < 9)):
            return 700
        elif ((age >= 9) & (age < 12)):
            return 800
        elif ((age >= 12) & (age < 15)):
            return 1000
        elif ((age >= 15) & (age < 19)):
            return 900
        elif ((age >= 19) & (age < 50)):
            return 800
        elif ((age >= 50) & (age < 65)):
            return 750
        elif ((age >= 65) & (age < 75)):
            return 700
        elif (age >= 75):
            return 700
    elif (sex == 2):
        if ((age >= 6) & (age < 9)):
            return 700
        elif ((age >= 9) & (age < 12)):
            return 800
        elif ((age >= 12) & (age < 15)):
            return 900
        elif ((age >= 15) & (age < 19)):
            return 800
        elif ((age >= 19) & (age < 50)):
            return 700
        elif ((age >= 50) & (age < 75)):
            return 800
        elif (age >= 75):
            return 800
    else:
      return 0

In [ ]:
df['SN_CA'] = df.apply(standard_CA, axis=1)

In [ ]:
def standard_PHOS(series):
    age = series["age"]
    sex = series["sex"]

    if ((age >= 1) & (age < 3)):
        return 450
    elif ((age >= 3) & (age < 6)):
        return 550
    elif (sex == 1):
        if ((age >= 6) & (age < 9)):
            return 600
        elif ((age >= 9) & (age < 19)):
            return 1200
        elif ((age >= 19) & (age < 75)):
            return 700
        elif (age >= 75):
            return 700
    elif (sex == 2):
        if ((age >= 6) & (age < 9)):
            return 550
        elif ((age >= 9) & (age < 19)):
            return 1200
        elif ((age >= 12) & (age < 75)):
            return 700
        elif (age >= 75):
            return 700
    else:
      return 0

In [ ]:
df['SN_PHOS'] = df.apply(standard_PHOS, axis=1)

In [ ]:
def standard_NA(series):
    age = series["age"]
    sex = series["sex"]

    if ((age >= 1) & (age < 3)):
        return 0.9
    elif ((age >= 3) & (age < 6)):
        return 1.0
    elif ((age >= 6) & (age < 9)):
        return 1.2
    elif ((age >= 9) & (age < 12)):
        return 1.4
    elif ((age >= 12) & (age < 65)):
        return 1.5
    elif ((age >= 65) & (age < 75)):
        return 1.3
    elif (age >= 75):
        return 1.1
    else:
      return 0

In [ ]:
df['SN_NA'] = df.apply(standard_NA, axis=1)

In [ ]:
def standard_K(series):
    age = series["age"]
    sex = series["sex"]

    if ((age >= 1) & (age < 3)):
        return 2.0
    elif ((age >= 3) & (age < 6)):
        return 2.3
    elif ((age >= 6) & (age < 9)):
        return 2.6
    elif ((age >= 9) & (age < 12)):
        return 3.0
    elif ((age >= 12) & (age < 75)):
        return 3.5
    elif (age >= 75):
        return 3.5
    else:
      return 0

In [ ]:
df['SN_K'] = df.apply(standard_K, axis=1)

In [ ]:
def standard_FE(series):
    age = series["age"]
    sex = series["sex"]

    if ((age >= 1) & (age < 6)):
        return 6
    elif (sex == 1):
        if ((age >= 6) & (age < 9)):
            return 9
        elif ((age >= 9) & (age < 12)):
            return 10
        elif ((age >= 12) & (age < 19)):
            return 14
        elif ((age >= 19) & (age < 65)):
            return 10
        elif ((age >= 65) & (age < 75)):
            return 9
        elif (age >= 75):
            return 9
    elif (sex == 2):
        if ((age >= 6) & (age < 9)):
            return 8
        elif ((age >= 9) & (age < 12)):
            return 10
        elif ((age >= 12) & (age < 15)):
            return 16
        elif ((age >= 15) & (age < 50)):
            return 14
        elif ((age >= 50) & (age < 75)):
            return 8
        elif (age >= 75):
            return 7
    else:
      return 0

In [ ]:
df['SN_FE'] = df.apply(standard_FE, axis=1)

In [ ]:
print(df[['age', 'sex', 'N_EN', 'SN_EN', 'N_PROT', 'SN_PROT', 'N_TDF', 'SN_TDF', 'N_WATER', 'SN_WATER', 'N_VITC', 'SN_VITC', 'N_B1', 'SN_B1', 'N_B2', 'SN_B2', 'N_NIAC', 'SN_NIAC', 'N_VA_RAE', 'SN_VA_RAE', 'N_CA', 'SN_CA', 'N_PHOS', 'SN_PHOS', 'N_NA', 'SN_NA', 'N_K', 'SN_K', 'N_FE', 'SN_FE']])

In [ ]:
# --- 표준영양소값과 개인별 영양소값의 차이를 구하기 ---

In [ ]:
df['Diff_EN'] = np.where(df['N_EN'] == df['SN_EN'], 0, df['N_EN']-df['SN_EN'])
df['Diff_PROT'] = np.where(df['N_PROT'] == df['SN_PROT'], 0, df['N_PROT']-df['SN_PROT'])
df['Diff_TDF'] = np.where(df['N_TDF'] == df['SN_TDF'], 0, df['N_TDF']-df['SN_TDF'])
df['Diff_WATER'] = np.where(df['N_WATER'] == df['SN_WATER'], 0, df['N_WATER']-df['SN_WATER'])
df['Diff_VITC'] = np.where(df['N_VITC'] == df['SN_VITC'], 0, df['N_VITC']-df['SN_VITC'])
df['Diff_B1'] = np.where(df['N_B1'] == df['SN_B1'], 0, df['N_B1']-df['SN_B1'])
df['Diff_B2'] = np.where(df['N_B2'] == df['SN_B2'], 0, df['N_B2']-df['SN_B2'])
df['Diff_NIAC'] = np.where(df['N_NIAC'] == df['SN_NIAC'], 0, df['N_NIAC']-df['SN_NIAC'])
df['Diff_VA_RAE'] = np.where(df['N_VA_RAE'] == df['SN_VA_RAE'], 0, df['N_VA_RAE']-df['SN_VA_RAE'])
df['Diff_CA'] = np.where(df['N_CA'] == df['SN_CA'], 0, df['N_CA']-df['SN_CA'])
df['Diff_PHOS'] = np.where(df['N_PHOS'] == df['SN_PHOS'], 0, df['N_PHOS']-df['SN_PHOS'])
df['Diff_NA'] = np.where(df['N_NA'] == df['SN_NA'], 0, df['N_NA']-df['SN_NA'])
df['Diff_K'] = np.where(df['N_K'] == df['SN_K'], 0, df['N_K']-df['SN_K'])
df['Diff_FE'] = np.where(df['N_FE'] == df['SN_FE'], 0, df['N_FE']-df['SN_FE'])

print(df[['N_EN', 'SN_EN', 'Diff_EN', 'N_PROT', 'SN_PROT', 'Diff_PROT', 'N_TDF', 'SN_TDF', 'Diff_TDF', 
          'N_WATER', 'SN_WATER', 'Diff_WATER', 'N_VITC', 'SN_VITC', 'Diff_VITC', 'N_B1', 'SN_B1', 'Diff_B1',
          'N_B2', 'SN_B2', 'Diff_B2', 'N_NIAC', 'SN_NIAC', 'Diff_NIAC', 'N_VA_RAE', 'SN_VA_RAE', 'Diff_VA_RAE',
          'N_CA', 'SN_CA', 'Diff_CA', 'N_PHOS', 'SN_PHOS', 'Diff_PHOS', 'N_NA', 'SN_NA', 'Diff_NA',
          'N_K', 'SN_K', 'Diff_K', 'N_FE', 'SN_FE', 'Diff_FE']])

In [ ]:
pd.set_option("max_columns", None)
print(df[['Diff_EN', 'Diff_PROT', 'Diff_TDF', 'Diff_WATER', 'Diff_VITC', 
          'Diff_B1', 'Diff_B2','Diff_NIAC', 'Diff_VA_RAE', 'Diff_CA', 
          'Diff_PHOS', 'Diff_NA', 'Diff_K', 'Diff_FE']])